In [1]:
#pip install kagglehub
#!pip install imblearn 
#!pip install imbalanced-learn

In [2]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import imblearn
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import time
print("Done")

Done


In [3]:
path = kagglehub.dataset_download("henrysue/online-shoppers-intention")
# print("Path to dataset files:", path)

In [4]:
dataset = pd.read_csv(f"{path}/online_shoppers_intention.csv")
print("Dataset Shape:", dataset.shape)
print("Dataset Preview:")
print(dataset.head())

Dataset Shape: (12330, 18)
Dataset Preview:
   Administrative  Administrative_Duration  Informational  \
0               0                      0.0              0   
1               0                      0.0              0   
2               0                      0.0              0   
3               0                      0.0              0   
4               0                      0.0              0   

   Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                     0.0               1                 0.000000   
1                     0.0               2                64.000000   
2                     0.0               1                 0.000000   
3                     0.0               2                 2.666667   
4                     0.0              10               627.500000   

   BounceRates  ExitRates  PageValues  SpecialDay Month  OperatingSystems  \
0         0.20       0.20         0.0         0.0   Feb                 1   
1         0.00

In [5]:
# checking for missing values
print("Missing Values:\n", dataset.isnull().sum())

Missing Values:
 Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64


In [6]:
label_encoder = LabelEncoder() # encoding our data
for column in dataset.select_dtypes(include=['object']).columns:
    dataset[column] = label_encoder.fit_transform(dataset[column])

In [7]:
X = dataset.drop(columns=['Revenue']) # establishing the target,y, vs the predictor cols
y = dataset['Revenue']
print(y)

0        False
1        False
2        False
3        False
4        False
         ...  
12325    False
12326    False
12327    False
12328    False
12329    False
Name: Revenue, Length: 12330, dtype: bool


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# splitting into train data and test data
# test is 30% of db

In [9]:
model = GaussianNB()  # predefined gaussian naive bayes model from sklearn lib
model.fit(X_train, y_train)


GaussianNB()

In [10]:
y_pred = model.predict(X_test) # predict


# performance metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.2f}")
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.2f}")

Accuracy: 0.85
Precision: 0.52
Recall: 0.53


In [11]:
# func for calculating mean and variance
def calculate_mean_variance(data):
    return np.mean(data, axis=0), np.var(data, axis=0)

In [12]:
# Gaussian probability density function
# naive bayes assumes normal distribution of data
def gaussian_pdf(x, mean, var):
    return (1 / np.sqrt(2 * np.pi * var)) * np.exp(-(x - mean) ** 2 / (2 * var))

In [13]:
# predction funct using bayes algorithm
def predict(X, threshold=0.7):
    predictions = []
    for i in range(len(X)):
        
        # likelihood for each class
        likelihood_false = np.log(p_false)
        likelihood_true = np.log(p_true)
        for j in range(X.shape[1]):  # shape[1] for the number of features
            likelihood_false += np.log(gaussian_pdf(X[i, j], mean_false[j], var_false[j]))
            likelihood_true += np.log(gaussian_pdf(X[i, j], mean_true[j], var_true[j]))
        
        # posterior probabilities (normalized)
        total_likelihood = np.exp(likelihood_false) + np.exp(likelihood_true)
        posterior_false = np.exp(likelihood_false) / total_likelihood
        posterior_true = np.exp(likelihood_true) / total_likelihood
        
        # threshold to decide the class
        if posterior_true >= threshold:
            predictions.append(1)  # "True" prediction
        else:
            predictions.append(0)  # "False"
    return predictions


In [14]:
# Preprocessing 
# encoding categorical features 
start = time.time()
label_encoder = LabelEncoder()
for column in dataset.select_dtypes(include=['object']).columns:
    dataset[column] = label_encoder.fit_transform(dataset[column])

# features and target
X = dataset.drop(columns=['Revenue'])
y = dataset['Revenue']

# SelectKBest to select the top 'k' features
k_best = SelectKBest(score_func=f_classif, k=3)
X_new = k_best.fit_transform(X, y)

# scaling the best k features obtained before
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)



In [15]:
# SMOTE for oversampling, the data is imbalanced, class True in minority in the target var
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Separate data by class (after oversampling)
X_train_false = X_train_res[y_train_res == 0]
X_train_true = X_train_res[y_train_res == 1]

In [16]:
# Prediction part
# calculating prior probabilities with class weight adjustment
# adjusting the priors (inversely proportional to class frequency)
class_imbalance_factor = len(y_train_res) / (2 * np.bincount(y_train_res))
p_false = class_imbalance_factor[0] / (class_imbalance_factor[0] + class_imbalance_factor[1])
p_true = class_imbalance_factor[1] / (class_imbalance_factor[0] + class_imbalance_factor[1])

# mean and variance for each class
mean_false, var_false = calculate_mean_variance(X_train_false)
mean_true, var_true = calculate_mean_variance(X_train_true)

# predict 
y_pred_manual = predict(X_test)

In [17]:
# performance metrics
accuracy = accuracy_score(y_test, y_pred_manual)
print(f"Accuracy: {accuracy:.2f}") #correct predictions
precision = precision_score(y_test, y_pred_manual) 
print(f"Precision: {precision:.2f}") #tp/(tp+fp)
recall = recall_score(y_test, y_pred_manual)
print(f"Recall: {recall:.2f}") #tp/(tp+fn)
f1 = f1_score(y_test, y_pred_manual) # harmonic mean bt precision and recall
print(f"F1 Score: {f1:.2f}")
print("Exec time : {} seconds".format(time.time() - start))

Accuracy: 0.87
Precision: 0.60
Recall: 0.55
F1 Score: 0.57
Exec time : 5.600075721740723 seconds
